# IMPLEMENTACIÓN MODELO ResNet

En este notebook se modificará el conocido modelo de ResNet para el objetivo del proyecto

## Importación de librerías necesarias

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import pickle
import mne
import numpy as np
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse
from sklearn.utils import shuffle, class_weight
import glob
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from custom_dataset import CustomDataset1 #Archivo necesario para cargar los dataloaders
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataloader de entrenamiento
with open('../dataloaders/dataloader_train_raw_coherence.pkl', 'rb') as f:
    dataloader_train = pickle.load(f)

# Dataloader de validación
with open('../dataloaders/dataloader_val_raw_coherence.pkl', 'rb') as f:
    dataloader_val = pickle.load(f)

## Codificación de etiquetas
Este fue un paso previo necesario para calcular los pesos de clase balanceados del modelo

In [7]:
train_labels = []
for data in dataloader_train:
    labels = data['class_label']
    train_labels.extend(labels.tolist())

# Convertir a arreglo NumPy
train_labels = np.array(train_labels)

# Codificar las etiquetas de clase como enteros
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(train_labels)

C:\Users\hernandez\Documents\TFG\Untitled Folder\custom_dataset.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  coherence_matrix = self.transform(coherence_matrix)
C:\Users\hernandez\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Definición del modelo

In [8]:

'''============================CONSTRUCCIÓN DE RED CONVOLUCIONAL=====================================

--> BLOQUE DE CUELLO DE BOTELLA:
Este es tun tipo de bloque residual que utiliza convoluciones 1x1 para crear el efecto 'cuello de
botella'. El uso de este bloque reduce el número de parámetros y multiplicaciones matriciales, 
aumentando de esta manera la profundidad y teniendo menos parámetros

--> CLASE RESNET MODIFICADO: 
Es la parte principal de la contrucción del modelo yse basa en la arquitectura de ResNet.

--> MODELO ENSAMBLADO: 
COmbinación de múltiples modelos ResNetModified para aprovechar la diversidad y la complementariedad
de varios modelos para mejorar el rendimiento y generalización.

'''

class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(BottleneckBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        identity = self.shortcut(x)

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += identity
        out = self.relu(out)
        return out


class ModifiedResNet(nn.Module):
    def __init__(self, block, layers, num_classes=3):
        super(ModifiedResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


class EnsembledModel(nn.Module):
    def __init__(self, num_models, block, layers, num_classes=3):
        super(EnsembledModel, self).__init__()
        self.models = nn.ModuleList([ModifiedResNet(block, layers, num_classes) for _ in range(num_models)])
        self.fc = nn.Linear(num_models * num_classes, num_classes)

    def forward(self, x):
        outputs = [model(x) for model in self.models]
        outputs = torch.stack(outputs, dim=1)
        outputs = outputs.view(outputs.size(0), -1)
        outputs = self.fc(outputs)
        return outputs


#==========================================INSTANCIA DEL MODELO=====================================

num_models = 5  # Número de modelos a ensamblar
model = EnsembledModel(num_models, BottleneckBlock, [2, 4, 6])
model.to(device)

# Definición de la función de pérdida y optimizador
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(encoded_labels), y=encoded_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

EnsembledModel(
  (models): ModuleList(
    (0): ModifiedResNet(
      (conv1): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BottleneckBlock(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

## Entrenamiento y validación del modelo

Se eligió realizar la validación tras cada época del entrenamiento para ver cómo este iba evolucionando y si había sido óptimo en cada paso para evitaer efectos como el sobreajuste.

In [12]:
train_accu = []
train_losses = []
eval_losses = []
eval_accu = []

epochs = 30
for epoch in range(1, epochs + 1):
    # ====================================Entrenamiento==============================================
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data in tqdm(dataloader_train):
        reduced_matrix = data['coherence_matrix'].float().to(device)
        age = data['age'].float().to(device)
        class_label = data['class_label'].squeeze().long().to(device)

        optimizer.zero_grad()

        age_expanded = age.unsqueeze(1).unsqueeze(2).repeat(1, 1, reduced_matrix.size(2), reduced_matrix.size(3))

        inputs = torch.cat((reduced_matrix, age_expanded), dim=1)

        outputs = model(inputs)
        loss = criterion(outputs, class_label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += class_label.size(0)
        train_correct += (predicted == class_label).sum().item()

    train_accuracy = 100.0 * train_correct / train_total
    train_loss /= len(dataloader_train)

    # ====================================Validación=================================================
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for data in tqdm(dataloader_val):
            reduced_matrix = data['coherence_matrix'].float().to(device)
            age = data['age'].float().to(device)
            class_label = data['class_label'].squeeze().unsqueeze(0).long().to(device)

            age_expanded = age.unsqueeze(1).unsqueeze(2).repeat(1, 1, reduced_matrix.size(2), reduced_matrix.size(3))

            inputs = torch.cat((reduced_matrix, age_expanded), dim=1)

            outputs = model(inputs)
            loss = criterion(outputs, class_label)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_total += class_label.size(0)
            test_correct += (predicted == class_label).sum().item()

    test_accuracy = 100.0 * test_correct / test_total
    test_loss /= len(dataloader_val)

    print(f"Epoch {epoch}/{epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%")
    print()


100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:03<00:00, 32.74it/s]


Epoch 1/30
Train Loss: 1.2687 | Train Accuracy: 43.02%
Test Loss: 1.0446 | Test Accuracy: 50.42%



100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:03<00:00, 32.63it/s]


Epoch 2/30
Train Loss: 1.0645 | Train Accuracy: 45.58%
Test Loss: 0.9442 | Test Accuracy: 54.62%



100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:03<00:00, 32.78it/s]


...

Epoch 28/30
Train Loss: 0.9946 | Train Accuracy: 47.50%
Test Loss: 0.9368 | Test Accuracy: 54.62%



100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:03<00:00, 32.03it/s]


Epoch 29/30
Train Loss: 0.9923 | Train Accuracy: 47.07%
Test Loss: 0.9343 | Test Accuracy: 56.30%



100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:03<00:00, 30.94it/s]

Epoch 30/30
Train Loss: 0.9901 | Train Accuracy: 46.75%
Test Loss: 0.9314 | Test Accuracy: 55.46%



In [19]:
test_loss = 0.0
test_correct = 0
test_total = 0

model = EnsembledModel(5, BottleneckBlock, [2, 4, 6])
model.to(device)
model.load_state_dict(torch.load('ResNet.pth'))

model.eval()

model.eval()
with torch.no_grad():
    for data in tqdm(dataloader_val):
        reduced_matrix = data['coherence_matrix'].float().to(device)
        age = data['age'].float().to(device)
        class_label = data['class_label'].squeeze().unsqueeze(0).long().to(device)

        age_expanded = age.unsqueeze(1).unsqueeze(2).repeat(1, 1, reduced_matrix.size(2), reduced_matrix.size(3))

        inputs = torch.cat((reduced_matrix, age_expanded), dim=1)

        outputs = model(inputs)
        loss = criterion(outputs, class_label)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        test_total += class_label.size(0)
        test_correct += (predicted == class_label).sum().item()
test_accuracy = 100.0 * test_correct / test_total
test_loss /= len(dataloader_val)

print('Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, test_accuracy))

100%|████████████████████████████████████████████████████████████████████████████████| 119/119 [00:06<00:00, 17.24it/s]

Test Loss: 0.9314, Accuracy: 55.46%


In [14]:
torch.save(model.state_dict(), 'ResNet.pth')